# Preprocess
- Thu thập data cho 2 tập valid và fake
- Link dataset: ```/mnt/ssd/dataset_cccd/```

# Train
- Thử nghiệm với model: resnet50


## resnet50

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, ConcatDataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
import torchmetrics
from lightning.pytorch.loggers import TensorBoardLogger
from datetime import datetime
from torchvision.transforms import v2 as transformsV2

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
MODEL_NAME = 'resnet50'


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((448, 448)),  
        transforms.RandomHorizontalFlip(),
        # transforms.RandomEqualize(),
        transforms.ColorJitter(brightness=0, contrast=1.5, saturation=0, hue=0),
        transforms.ToTensor(),
        # translate, transform, flip, blur
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize((448, 448)), 
        # transforms.RandomVerticalFlip(),
        # transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0, contrast=1.5, saturation=0, hue=0),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

# data_dir = '/mnt/ssd/dataset_cccd/train_ekyc_moire_or_not/dataset'
# full_dataset = ImageFolder(data_dir, transform=data_transforms['train'])


# train_size = int(0.8 * len(full_dataset))
# val_size = len(full_dataset) - train_size
# train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_ds = ImageFolder('/mnt/ssd/ekyc_myanmar/project_fake_profile/dataset_train', transform=data_transforms['train'])

# concatenate train_ds_no_padding AND train_ds_with_padding
train_data = ConcatDataset([train_ds])


val_dataset, train_dataset = random_split(train_data, [0.2, 0.8])
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


class ResNetClassifier(pl.LightningModule):
    def __init__(self, num_classes=2):
        super(ResNetClassifier, self).__init__()
        self.model = models.resnet50(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)  # 2 classes: fake, valid
        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_classes)
        self.f1_score = torchmetrics.F1Score(task="multiclass", num_classes=num_classes)
        self.confusion_matrix = torchmetrics.ConfusionMatrix(task='multiclass', num_classes=num_classes)
        self.wrong_preds_images = {i: [] for i in range(num_classes)}
        self.num_classes = num_classes
        self.example_input_array = torch.randn(1, 3, 448, 448)

 
    def forward(self, x):
        return self.model(x)


    def configure_optimizers(self):
        optimizer = optim.SGD(self.model.parameters(), lr=0.002, momentum=0.95)
        # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, threshold=0.01, patience=8)
        return  {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}


    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.forward(inputs)
        loss = self.criterion(outputs, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True) 
        accuracy = self.accuracy(outputs, labels)
        self.log("train_acc", accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "outputs_labels": outputs, "ground_truth_labels": labels}


    def on_training_epoch_end(self):
        pass
        


    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.forward(inputs)
        loss = self.criterion(outputs, labels)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True) 
        self.accuracy(outputs, labels)
        self.log("val_acc", self.accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss


    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        
        preds = torch.argmax(outputs, dim=1)

        # self.log('test_loss', loss)
        # log step metric
        self.accuracy(preds, labels)
        self.confusion_matrix(preds, labels)

        self.log('test_acc', self.accuracy)

         # Collect wrong predictions
        wrong_indices = torch.where(preds != labels)[0]
        for i in wrong_indices:
            true_label = labels[i].item()
            self.wrong_preds_images[true_label].append(inputs[i])


    def on_test_epoch_end(self) -> None:
        # Concatenate all wrong predictions
        for class_id in range(self.num_classes):
            if self.wrong_preds_images[class_id]:
                wrong_images = torch.stack(self.wrong_preds_images[class_id])
                grid = torchvision.utils.make_grid(wrong_images)
                self.logger.experiment.add_image(f'wrong_predictions_class_{class_id}', grid, self.current_epoch)
        

        val_cm = self.confusion_matrix.compute()
        # self.log('test_confusion_matrix', val_cm)
        print(f'Validation Confusion Matrix:\n{val_cm}')
        self.confusion_matrix.reset()

        return super().on_test_epoch_end()


    def predict_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.forward(inputs)
        predictions = torch.argmax(outputs, dim=1)
        return predictions



checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=f'checkpoints/resnet50_fake_or_not_ckpt_{current_time}',
    filename='{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}',
    save_top_k=3,
    mode='min'
)

early_stop_callback = EarlyStopping(
    monitor="val_acc", 
    min_delta=0.00,
    patience=15,
    verbose=False,
    mode="max"
)

logger = TensorBoardLogger(
    save_dir='.',
    name=f'logs_{MODEL_NAME}'
)

# Trainer
trainer = Trainer(
    max_epochs=100,
    accelerator='gpu', 
    precision=16,
    callbacks=[checkpoint_callback, early_stop_callback],
    logger=logger  # Add the logger to the Trainer
)

model = ResNetClassifier()
# trainer.fit(model, train_loader, val_loader)

/mnt/ssd/jason/anaconda3/envs/p_light_3_11/lib/python3.11/site-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/mnt/ssd/jason/anaconda3/envs/p_light_3_11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/ssd/jason/anaconda3/envs/p_light_3_11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=R

# metrics

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torchvision.transforms import v2 as transformsV2
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
import torchmetrics
from lightning.pytorch.loggers import TensorBoardLogger


test_transform =  transforms.Compose([
        transforms.Resize((448, 448)),
        # transforms.ColorJitter(brightness=0, contrast=1.5, saturation=0, hue=0),
        transforms.ToTensor(),
        # transforms.GaussianBlur(kernel_size=(5, 5), sigma=5.),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])

    ]) 
test_dataset_dir = '/mnt/ssd/dataset_cccd/project_fake_profile/dataset_test'
test_dataset = ImageFolder(test_dataset_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


model = ResNetClassifier()

# Trainer
trainer = Trainer(
    # max_epochs=100,
    accelerator='cuda', 
    # precision=16,
    # callbacks=[checkpoint_callback, early_stop_callback],
    # logger=logger  # Add the logger to the Trainer
)

trainer.test(
    model,
    ckpt_path='/mnt/ssd/dataset_cccd/project_fake_profile/checkpoints/resnet50_fake_or_not_ckpt_20240718_135446/epoch=08-val_loss=0.11-val_acc=0.98.ckpt',
    dataloaders=[test_loader]
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /mnt/ssd/ekyc_myanmar/project_fake_profile/checkpoints/resnet50_fake_or_not_ckpt_20240718_135446/epoch=08-val_loss=0.11-val_acc=0.98.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /mnt/ssd/ekyc_myanmar/project_fake_profile/checkpoints/resnet50_fake_or_not_ckpt_20240718_135446/epoch=08-val_loss=0.11-val_acc=0.98.ckpt


Testing DataLoader 0: 100%|██████████| 88/88 [00:18<00:00,  4.83it/s]Validation Confusion Matrix:
tensor([[1276,  418],
        [ 141,  951]], device='cuda:0')
Testing DataLoader 0: 100%|██████████| 88/88 [00:38<00:00,  2.26it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7993538975715637
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.7993538975715637}]

In [ ]:
! unzip '/mnt/ssd/ekyc_myanmar/train_ekyc_moire_or_not/Device-20240621T030613Z-001.zip'

# export model to ONNX

In [3]:
model = ResNetClassifier.load_from_checkpoint('/mnt/ssd/dataset_cccd/project_fake_profile/checkpoints/resnet50_fake_or_not_ckpt_20240716_173045_norm_485/epoch=12-val_loss=0.10-val_acc=0.97.ckpt')
filepath = "resnet50_fake_or_not_240718_epoch_12_val_loss_010_val_acc_097.onnx"
model.to_onnx(filepath, export_params=True)

/mnt/ssd/jason/anaconda3/envs/p_light_3_11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/ssd/jason/anaconda3/envs/p_light_3_11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


: 

In [ ]:
!unzip -q '/mnt/ssd/jason/techainer_projects/images/front_good_rotate-20240627T022031Z-001.zip' -d /mnt/ssd/jason/techainer_projects/images/
!rm '/mnt/ssd/jason/techainer_projects/images/front_good_rotate-20240627T022031Z-001.zip'

 # OCR False case > 20242404-0205

In [ ]:
!zip Lib-Card-Validation-Training-dataset.zip -r /mnt/ssd/ekyc_myanmar/Lib-Card-Validation-Training/dataset